In [1]:
import numpy as np 
import cv2

In [2]:
def calculate_mse(image1, image2):
    error = np.mean((image1 - image2) ** 2)
    return error

#define the network with diff parameters and helps deploying caffe model
#Use to predict new unseen data
#Stores cluster centerpoints in numpy format

print("loading models.....")
net = cv2.dnn.readNetFromCaffe('colorization_deploy_v2.prototxt','colorization_release_v2.caffemodel')
pts = np.load('pts_in_hull.npy')  #contains set of color points of LAB 

loading models.....


In [3]:
# Converts string name layer into int
class8 = net.getLayerId("class8_ab")  # color points
conv8 = net.getLayerId("conv8_313_rh")  # colorization parameters
pts = pts.transpose().reshape(2, 313, 1, 1)

net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype='float32')]

image = cv2.imread('lion.jpg')
scaled = image.astype("float32") / 255.0
lab = cv2.cvtColor(scaled, cv2.COLOR_BGR2LAB)

resized = cv2.resize(lab, (224, 224))
L = cv2.split(resized)[0]
L -= 50

net.setInput(cv2.dnn.blobFromImage(L))
ab = net.forward()[0, :, :, :].transpose((1, 2, 0))

ab = cv2.resize(ab, (image.shape[1], image.shape[0]))

L = cv2.split(lab)[0]
colorized = np.concatenate((L[:, :, np.newaxis], ab), axis=2)

colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
colorized = np.clip(colorized, 0, 1)  # Remove scaling factor

colorized = (255 * colorized).astype("uint8")

cv2.imshow("Original", image)
cv2.imshow("Colorized", colorized)
cv2.waitKey(0)

mse = calculate_mse(image, colorized)

print("Mean Squared Error (MSE):", mse)


Mean Squared Error (MSE): 75.67160280626013
